<a href="https://colab.research.google.com/github/jkranyak/project_3/blob/main/project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Collect and Prepare the Dataset

You'll need a dataset of skin growth images with labels indicating the type of growth.
The data should be preprocessed (e.g., resizing, normalization) to be suitable for training a model.



Collect and Prepare the Dataset
The first step in building an AI model to assess types of skin growths through photographs is to collect and prepare a dataset. This process involves several key activities:

SStep 1: API Access
API Documentation: First, review the ISIC Archive API documentation. It provides detailed information on how to use the API, including authentication, endpoint descriptions, and examples.
Authentication: If required, obtain an API key or access token for authentication. The process should be outlined in the documentation.

In [1]:
!pip install isic-cli


Step 2: Query the Dataset
Endpoints: Use the API endpoints to query the dataset. Common operations include listing available images, retrieving image metadata, and downloading images.
Filtering: Utilize query parameters to filter the dataset based on your criteria, such as diagnosis, image type, or other metadata.

In [2]:
!pip install isic-cli


In [21]:
# List available collections
!isic collection list

If you have been granted special permissions, logging in with `isic user login` might return more data.

┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ ID  ┃ Name                                          ┃ Public ┃ Pinned ┃ Locked ┃ DOI             ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 249 │ BCN20000                                      │ True   │ False  │ False  │ None            │
│ 61  │ Challenge 2016: Test                          │ True   │ True   │ True   │ None            │
│ 74  │ Challenge 2016: Training                      │ True   │ True   │ True   │ None            │
│ 69  │ Challenge 2017: Test                          │ True   │ True   │ True   │ None            │
│ 60  │ Challenge 2017: Training                      │ True   │ True   │ True   │ None            │
│ 71  │ Challenge 2017: Validation                    │ True   │ True   │ True   │ None

Step 3: Download Images and Metadata
Batch Processing: Write a script to automatically download batches of images and their associated metadata.

Data Storage will utilize wounting a drive in google and creating files for the train, test, and validation images.

In [22]:
! isic user login

visit the following url in a browser:
https://api.isic-archive.com/oauth/authorize?response_type=code&client_id=RpCzc4hFjv5gOJdM2DM2nBdokOviOh5ne63Tpn7Q&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=wb9ForGyuJMEKvoCodpQ0nsbcYAgEl&code_challenge=2cNQFvyscLm7QrAj_m0ir0PF5NU3fNj4Do-JZVq1V0U&code_challenge_method=S256
enter the code shown in your browser: DikQiK3r9iwixAkfjODPWj0fVyxA3d
Success!


In [23]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [1]:
!mkdir -p "/content/drive/My Drive/ISIC_2018/training/"
!mkdir -p "/content/drive/My Drive/ISIC_2018/validation/"
!mkdir -p "/content/drive/My Drive/ISIC_2018/test/"

# Download the Challenge 2018: Task 1-2: Training set (ID 63)
!isic image download --search 'collection_id:63' '/content/drive/My Drive/ISIC_2018/training/'

# Download the Challenge 2018: Task 1-2: Validation set (ID 62)
!isic image download --search 'collection_id:62' '/content/drive/My Drive/ISIC_2018/validation/'

# Download the Challenge 2018: Task 1-2: Test set (ID 64)
!isic image download --search 'collection_id:64' '/content/drive/My Drive/ISIC_2018/test/'



The following unexpected error occurred while attempting your operation:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/isic_cli/cli/__init__.py", line 183, in main
    cli()
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 783, in invoke
    return __callback(*args, **kwargs)
  Fi

In [3]:
response = requests.get(f"{api_base_url}/images?collectionId=60")
if response.status_code == 200:
    images = response.json()
    # Process or download images
else:
    print("Failed to retrieve images:", response.status_code)


2. Dataset Preparation

Image Preprocessing: Preprocess the images to a consistent size and format. normalize the image data (scaling pixel values to a range of 0 to 1).
Data Augmentation: To increase the size of dataset and improve the model's robustness, applying data augmentation techniques such as rotation, zoom, flip, and shift.
Labeling: Ensure each image is labeled with the correct type of skin growth. This might involve categorizing images by condition (e.g., benign, malignant, melanoma, etc.).

In [10]:
!pip install tensorflow


In [11]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory


Define preprocess Functions

In [12]:
def preprocess_data(training_path, validation_path, test_path, img_size, batch_size):
    """
    Preprocess the data: Load, resize, normalize, and optionally augment the images.

    Parameters:
    - training_path: Path to the training data directory.
    - validation_path: Path to the validation data directory.
    - test_path: Path to the test data directory.
    - img_size: The target size of the images (width, height).
    - batch_size: The size of the batches of data.

    Returns:
    - A tuple of (training_data, validation_data, test_data).
    """
    # Rescaling pixel values from [0, 255] to [0, 1] for neural network to process
    rescale = 1./255

    # Preparing the training data with data augmentation
    train_datagen = ImageDataGenerator(
        rescale=rescale,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Preparing the validation and test data (no data augmentation, only rescaling)
    val_test_datagen = ImageDataGenerator(rescale=rescale)

    # Loading and iterating directories for training, validation, and test sets
    train_data = train_datagen.flow_from_directory(
        training_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary' # or 'categorical' based on your problem
    )

    validation_data = val_test_datagen.flow_from_directory(
        validation_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary' # or 'categorical'
    )

    test_data = val_test_datagen.flow_from_directory(
        test_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary' # or 'categorical'
    )

    return train_data, validation_data, test_data

Preprocess

In [14]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory


In [15]:
def preprocess_data(training_path, validation_path, test_path, img_size, batch_size):
    """
    Preprocess the data: Load, resize, normalize, and optionally augment the images.

    Parameters:
    - training_path: Path to the training data directory.
    - validation_path: Path to the validation data directory.
    - test_path: Path to the test data directory.
    - img_size: The target size of the images (width, height).
    - batch_size: The size of the batches of data.

    Returns:
    - A tuple of (training_data, validation_data, test_data).
    """
    # Rescaling pixel values from [0, 255] to [0, 1] for neural network to process
    rescale = 1./255

    # Preparing the training data with data augmentation
    train_datagen = ImageDataGenerator(
        rescale=rescale,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Preparing the validation and test data (no data augmentation, only rescaling)
    val_test_datagen = ImageDataGenerator(rescale=rescale)

    # Loading and iterating directories for training, validation, and test sets
    train_data = train_datagen.flow_from_directory(
        training_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary' # or 'categorical' based on your problem
    )

    validation_data = val_test_datagen.flow_from_directory(
        validation_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary' # or 'categorical'
    )

    test_data = val_test_datagen.flow_from_directory(
        test_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary' # or 'categorical'
    )

    return train_data, validation_data, test_data

preprocess dataset


In [16]:
training_path = '/content/drive/My Drive/ISIC_images/training/'
validation_path = '/content/drive/My Drive/ISIC_images/validation/'
test_path = '/content/drive/My Drive/ISIC_images/test/'

# Example size and batch size
img_size = (224, 224)
batch_size = 32

train_data, validation_data, test_data = preprocess_data(training_path, validation_path, test_path, img_size, batch_size)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


Develop the Image Classification Model

here we will choose a Convolutional Neural Network, as it is able to process images effectively


Define the model

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [18]:
def build_model(img_size):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Use 'softmax' for multiclass classification
    ])

    return model

compile the model

In [19]:
model = build_model(img_size)

model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Use 'categorical_crossentropy' for multiclass classification
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

Train the model

In [20]:
history = model.fit(
    train_data,
    validation_data=validation_data,
    epochs=10,  # You can adjust the number of epochs based on your observations
    verbose=1
)

ValueError: Asked to retrieve element 0, but the Sequence has length 0

Evaluate the model

In [ ]:
test_loss, test_acc = model.evaluate(test_data, verbose=2)
print(f"Test accuracy: {test_acc}")

3. Implement the NLP Component

Utilize a pre-trained model like GPT-3 for generating explanations or advice based on the classification results.
This could involve sending a prompt to the model that includes the classification result and asking it to generate a user-friendly explanation.

4. Integrate APIs

If using OpenAI's GPT-3, you'll interact with it through its API, sending requests with the classification results and receiving generated text.
If your application needs to store or retrieve data, consider integrating a database and accessing it through a REST API.


5. Build the Application Logic

Develop a user interface where users can upload photos of skin growths.
After uploading, the photo is sent to the image classification model to determine the type of growth.
The classification result is used to generate a prompt for the ChatGPT-like model, which then creates an informative response.
Display the response to the user along with any additional advice or recommendations.